In [16]:
import yaml, os
from pinecone import Pinecone
from pinecone import ServerlessSpec
from llama_index.schema import TextNode
from llama_index.llms import AzureOpenAI, OpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.vector_stores import MetadataFilters
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.text_splitter import TokenTextSplitter
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.vector_stores import PineconeVectorStore, MetadataFilters
from llama_index.indices.vector_store.retrievers import VectorIndexAutoRetriever
from llama_index import ServiceContext, load_index_from_storage, StorageContext, VectorStoreIndex

# Yet to Complete

# Configure LLMs and VDB

In [2]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
api_key = credentials["PINECONE_API_KEY"]
pc = Pinecone(api_key=api_key)

In [4]:
llm_flag = 'DIRECT'

embedding_llm = HuggingFaceEmbedding(
                                    model_name="BAAI/bge-small-en-v1.5",
                                    device='mps'
                                    )

if llm_flag == 'AZURE':
    llm=AzureOpenAI(
                    model=credentials['AZURE_ENGINE'],
                    api_key=credentials['AZURE_OPENAI_API_KEY'],
                    deployment_name=credentials['AZURE_DEPLOYMENT_ID'],
                    api_version=credentials['AZURE_OPENAI_API_VERSION'],
                    azure_endpoint=credentials['AZURE_OPENAI_API_BASE'],
                    temperature=0.3
                    )
    
    chat_llm = LLMPredictor(llm)
else:
    chat_llm = OpenAI(
                    api_key=credentials['DEMO_OPENAI_API_KEY'],
                    temperature=0.3
                    )

text_splitter = TokenTextSplitter(
                                separator=" ",
                                chunk_size=1024,
                                chunk_overlap=20,
                                backup_separators=["\n"]
                                )

if llm_flag == 'AZURE':
    service_context = ServiceContext.from_defaults(
                                                    text_splitter=text_splitter,
                                                    # prompt_helper=prompt_helper,
                                                    embed_model=embedding_llm,
                                                    llm_predictor=chat_llm
                                                    )
else:
    service_context = ServiceContext.from_defaults(
                                                    text_splitter=text_splitter,
                                                    # prompt_helper=prompt_helper,
                                                    embed_model=embedding_llm,
                                                    llm=chat_llm
                                                    )

set_global_service_context(service_context)

In [5]:
# Dimensions are for text-embedding-ada-002
try:
    pc.create_index(
                    name="advanced-rag-experiments",
                    dimension=384,                  # Replace with your model dimensions
                    metric="euclidean",             # Replace with your model metric
                    spec=ServerlessSpec(
                                        cloud="aws",
                                        region="us-west-2"
                                        ) 
                    )
except Exception as e:
    print(e)
    pass

(409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '9ea56095fac1a503258c39cb8b43b398', 'Date': 'Thu, 18 Jan 2024 15:09:57 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}



In [6]:
pinecone_index = pc.Index("advanced-rag-experiments")

In [ ]:
# https://github.com/run-llama/llama_index/blob/main/docs/examples/vector_stores/pinecone_auto_retriever.ipynb